# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다
2. 워드 임베딩 모델을 학습시킨다
3. 단어 유사도가 0.8 이상인 A, B를 랜덤 추출한다
4. A, B와 대응되는 C를 추출한다
5. D를 입력받는다

=>
A:B = C:D 관계에 대응하는 D를 찾는 게임을 만든다
ex) A: 산, B: 바다, C: 나무, D: 물

**<출력 예시>**

관계 [ 수긍 : 추락 = 대사관 : ? ] <br>
모델이 예측한 가장 적합한 단어 : 잠입 <br>
당신의 답변과 모델 예측의 유사도 : 0.34 <br>
아쉽네요. 더 생각해보세요.

In [1]:
# !pip install eunjeon

In [ ]:
# !pip install mecab-python3_ko_0.92_msvc-cp38-win_amd64.whl

ERROR: Invalid wheel filename (wrong number of parts): 'mecab-python3_ko_0.92_msvc-cp38-win_amd64'


In [3]:
# from Korpora import Korpora
# Korpora.fetch("kcbert", force_download=True)

In [4]:
import os
from Korpora import Korpora

korpora_path = os.path.expanduser("~\\Korpora\\namuwikitext\\namuwikitext_20200302.dev")
texts = []

with open(korpora_path, 'r', encoding='utf-8') as f:
    for line in f:
        texts.append(line.strip())

In [7]:
from konlpy.tag import Okt, Kkma
import re
from tqdm import tqdm
import pickle

preprocess_data = []
kkma = Kkma()
ko_stopwords = ['은', '는', '이', '가', '을', '를', '과', '와', '들', '도', '부터', '까지', '에', '나', '너', '그', '걔', '얘', '로서', '때문', '병림픽', '때문', '경우', '이후']

for sentence in tqdm(texts):
    cleaned_sentence = re.sub(r'[^가-힣\s]', '', sentence)
    nouns = kkma.nouns(cleaned_sentence)
    filtered_nouns = [word for word in nouns if word not in ko_stopwords and len(word) > 1]
    if filtered_nouns:
        preprocess_data.append(filtered_nouns)

# 전처리 데이터 저장
with open("preprocess_data.pkl", "wb") as f:
    pickle.dump(preprocess_data, f)

print("전처리 완료 및 저장됨")

print(preprocess_data[:5])

100%|██████████| 153605/153605 [53:32<00:00, 47.82it/s]  


전처리 완료 및 저장됨
[['에일', '에일리언', '리언'], ['에이', '에이리언', '주년', '공개', '단편', '목성형', '행성', '우주선', '하나', '폭발', '동강', '직전', '탈출', '탈출정', '탑승', '인원', '한명', '체스트', '버스터', '바람', '몰살', '줄거리', '스포일러', '혼수', '혼수상태', '상태', '산소', '산소마스크', '마스크', '사람', '쿠키', '쿠키영상', '영상', '생채기', '의식', '주변', '난장판'], ['참고', '우연', '일치', '트라', '단어', '자체', '에이', '에이리언', '커버넌트', '유사'], ['개요'], ['에서', '롤플레', '게임', '모바일', '출시', '로도']]


In [8]:
from gensim.models import Word2Vec, FastText

with open("preprocess_data.pkl", "rb") as f:
    preprocess_data = pickle.load(f)

model = FastText(
    sentences=preprocess_data, 
    vector_size=100,
    window=5,
    min_count=1,
    sg=1  
)

model.save("namu_fasttext_dev.model")

In [9]:
vocab = model.wv.index_to_key
print(len(vocab))
print(vocab[:5])

140078
['자신', '사람', '정도', '사용', '등장']


In [15]:
import random

def get_words(threshold=0.8):
        for _ in range(10000):
                a = random.choice(vocab)
                b = random.choice(vocab)
                if model.wv.similarity(a, b) >= threshold and a != b:
                        return a, b
        return None, None

def find_word_game(threshold=0.8):
        a, b = get_words(threshold=threshold)
        if a is None or b is None:
                print("a, b를 찾을 수 없습니다.")
                return

        c = random.choice(vocab)
        print(f"관계 [ {a} : {b} = {c} : ? ]")
    
        user_guess = input("예상 단어 입력").strip()

        d_answer = model.wv.most_similar(positive=[b, c], negative=[a], topn=1)[0][0]

        similarity = model.wv.similarity(user_guess, d_answer)

        print(f"모델이 예측한 가장 적합한 단어: '{d_answer}'")
        print(f"당신의 답변과 모델 예측의 유사도: {user_guess}, {similarity:.2f}")
        if similarity > 0.8:
                print("정답에 가깝습니다.")
        else:
                print("아쉽네요. 더 생각해보세요.")

find_word_game() 

관계 [ 효복 : 요마 = 박스오피스 : ? ]
모델이 예측한 가장 적합한 단어: '오피스'
당신의 답변과 모델 예측의 유사도: 영화, 0.52
아쉽네요. 더 생각해보세요.
